<a href="https://colab.research.google.com/github/Maria-lin/How-to-make-notebook-in-dataiku/blob/main/lesfonctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

def repartition_requetes(df, date_x, colonne_date='date', colonne_circuit='circuit'):
    """
    Calcule pour chaque circuit à une date donnée :
    - le nombre de requêtes
    - le total des requêtes pour cette date (nb_requetes_max)
    - le pourcentage de chaque circuit
    """
    # Filtrer les données sur la date demandée
    df_filtre = df[df[colonne_date] == date_x]

    # Si aucune donnée pour cette date, retourner un df vide
    if df_filtre.empty:
        return pd.DataFrame(columns=[colonne_date, colonne_circuit, 'nb_requetes', 'nb_requetes_max', 'pourcentage'])

    # Compter le nombre de requêtes par circuit
    counts = (
        df_filtre.groupby(colonne_circuit)
        .size()
        .reset_index(name='nb_requetes')
        .sort_values(by='nb_requetes', ascending=False)
    )

    # Calcul du total pour la date
    total_requetes = counts['nb_requetes'].sum()

    # Ajouter les colonnes demandées
    counts['nb_requetes_max'] = total_requetes
    counts['pourcentage'] = (counts['nb_requetes'] / total_requetes * 100).round(1)
    counts.insert(0, colonne_date, date_x)

    return counts.reset_index(drop=True)


In [2]:
import pandas as pd

def repartition_flux_par_apiuri(df, en_date, colonne_date='date', colonne_circuit='circuit', colonne_apiuri='apiURI'):
    """
    Calcule pour le circuit RVM et une date donnée :
    - le nombre de flux par apiURI
    - le pourcentage de ces flux par rapport au total RVM de cette date
    - affiche le tout trié par pourcentage décroissant
    """
    # Filtrer sur la date et le circuit RVM
    df_filtre = df[(df[colonne_date] == en_date) & (df[colonne_circuit] == 'RVM')]

    # Si aucun flux trouvé, retourner un df vide
    if df_filtre.empty:
        return pd.DataFrame(columns=[colonne_date, colonne_circuit, colonne_apiuri, 'nb_flux', 'pourcentage'])

    # Remplacer les apiURI manquants par 'NA'
    df_filtre[colonne_apiuri] = df_filtre[colonne_apiuri].fillna('NA')

    # Calcul du nombre de flux par apiURI
    counts = (
        df_filtre.groupby(colonne_apiuri)
        .size()
        .reset_index(name='nb_flux')
        .sort_values(by='nb_flux', ascending=False)
    )

    # Calcul du total
    total_flux = counts['nb_flux'].sum()

    # Ajout du pourcentage
    counts['pourcentage'] = (counts['nb_flux'] / total_flux * 100).round(2)

    # Ajout des colonnes date et circuit
    counts.insert(0, colonne_circuit, 'RVM')
    counts.insert(0, colonne_date, en_date)

    # Tri par pourcentage décroissant
    counts = counts.sort_values(by='pourcentage', ascending=False).reset_index(drop=True)

    return counts
